## Content

[1. Import libraries](#Import_libraries)<br>
[2. Import data](#Import_data)<br>
[3.Data cleaning](#Data_cleaning)<br>
[- Removing lines with sold cars](#Removing_lines_with_sold_cars)<br>
[- Shifting data with missing columns 1 column to the right](#Shifting_data_with_missing_columns_1_column_to_the_right)<br>
[- Removing extra spaces](#Removing_extra_spaces)<br>
[- Shifting data on right columns](#Shifting_data_on_right_columns)<br>
[- Split data in "auto_engine" into 3 columns](#Split_auto_engine)<br>
[- Formating columns (Dtype, creating new columns, etc.)](#Formating_columns)<br>

<a id='Import_libraries'></a>
<h2>Import libraries</h2>

In [889]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics 
from sklearn.metrics import mean_squared_error

<a id='Import_data'></a>
## Import data

In [890]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2159 entries, 0 to 2130
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Type               2159 non-null   object
 1   auto_price         2159 non-null   object
 2   auto_year          2159 non-null   object
 3   auto_owners        2159 non-null   object
 4   auto_guarantee     2159 non-null   object
 5   auto_mileage       2159 non-null   object
 6   auto_engine        2159 non-null   object
 7   auto_transmission  2159 non-null   object
 8   auto_drive_unit    2159 non-null   object
 9   auto_wheel         2159 non-null   object
 10  auto_carcase       2159 non-null   object
 11  auto_color         2159 non-null   object
 12  link               2159 non-null   object
 13  guarantee          2159 non-null   int64 
dtypes: int64(1), object(13)
memory usage: 253.0+ KB


We see that there is a lot of data without info on car price. Cars with empty 'auto_price' column were sold, but information about them was not removed from the website.

<a id='Data_cleaning'></a>
## Data cleaning

<a id='Removing_lines_with_sold_cars'></a>
### Removing lines with sold cars 

In [891]:
data = data.dropna(subset=['auto_price'])

In [892]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2159 entries, 0 to 2130
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Type               2159 non-null   object
 1   auto_price         2159 non-null   object
 2   auto_year          2159 non-null   object
 3   auto_owners        2159 non-null   object
 4   auto_guarantee     2159 non-null   object
 5   auto_mileage       2159 non-null   object
 6   auto_engine        2159 non-null   object
 7   auto_transmission  2159 non-null   object
 8   auto_drive_unit    2159 non-null   object
 9   auto_wheel         2159 non-null   object
 10  auto_carcase       2159 non-null   object
 11  auto_color         2159 non-null   object
 12  link               2159 non-null   object
 13  guarantee          2159 non-null   int64 
dtypes: int64(1), object(13)
memory usage: 253.0+ KB


There are also some empty cells in 'auto_color' column.
Actually, data on some cars were shifted 1 column to the left as those cars has empty 'auto_guarantee' column (which means that cars doesn't have guarantee).

<a id='Shifting_data_with_missing_columns_1_column_to_the_right'></a>
### Shifting data with missing guarantee 1 column to the right

In [893]:
data['guarantee'] = 1
#data
data.loc[data['auto_color'].isnull(), ['guarantee']] = 0

In [894]:
data_to_shift = data.loc[data['guarantee'] == 0]
data = data.drop(data[data['guarantee'] == 0].index)
data_to_shift.shape, data.shape

((0, 14), (2159, 14))

In [895]:
data_to_shift

,Type,auto_price,auto_year,auto_owners,auto_guarantee,auto_mileage,auto_engine,auto_transmission,auto_drive_unit,auto_wheel,auto_carcase,auto_color,link,guarantee


In [896]:
data_to_shift['auto_color'] = data_to_shift['auto_carcase']
data_to_shift['auto_carcase'] = data_to_shift['auto_wheel']
data_to_shift['auto_wheel'] = data_to_shift['auto_drive_unit']
data_to_shift['auto_drive_unit'] = data_to_shift['auto_transmission']
data_to_shift['auto_transmission'] = data_to_shift['auto_engine']
data_to_shift['auto_engine'] = data_to_shift['auto_mileage']
data_to_shift['auto_mileage'] = data_to_shift['auto_year']
data_to_shift['auto_year'] = data_to_shift['auto_guarantee']
data_to_shift['auto_guarantee'] = "Нет гарантии"

In [897]:
data_to_shift.head()

,Type,auto_price,auto_year,auto_owners,auto_guarantee,auto_mileage,auto_engine,auto_transmission,auto_drive_unit,auto_wheel,auto_carcase,auto_color,link,guarantee


In [898]:
data_to_shift.shape

(0, 14)

In [899]:
data = pd.concat([data, data_to_shift])

In [900]:
data.shape

(2159, 14)

In [901]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2159 entries, 0 to 2130
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Type               2159 non-null   object
 1   auto_price         2159 non-null   object
 2   auto_year          2159 non-null   object
 3   auto_owners        2159 non-null   object
 4   auto_guarantee     2159 non-null   object
 5   auto_mileage       2159 non-null   object
 6   auto_engine        2159 non-null   object
 7   auto_transmission  2159 non-null   object
 8   auto_drive_unit    2159 non-null   object
 9   auto_wheel         2159 non-null   object
 10  auto_carcase       2159 non-null   object
 11  auto_color         2159 non-null   object
 12  link               2159 non-null   object
 13  guarantee          2159 non-null   int64 
dtypes: int64(1), object(13)
memory usage: 253.0+ KB


In [902]:
data[data['auto_color'].isnull()]['link'].unique()

array([], dtype=object)

In [903]:
data_shift_1 = data[data['auto_color'].isnull()]
data = data.drop(data[data['auto_color'].isnull()].index)
data.shape, data_shift_1.shape

((2159, 14), (0, 14))

In [904]:
data_shift_1

,Type,auto_price,auto_year,auto_owners,auto_guarantee,auto_mileage,auto_engine,auto_transmission,auto_drive_unit,auto_wheel,auto_carcase,auto_color,link,guarantee


In [905]:
data_shift_1['auto_color'] = data_shift_1['auto_carcase']
data_shift_1['auto_carcase'] = data_shift_1['auto_wheel']
data_shift_1['auto_wheel'] = data_shift_1['auto_drive_unit']
data_shift_1['auto_drive_unit'] = data_shift_1['auto_transmission']
data_shift_1['auto_transmission'] = data_shift_1['auto_engine']
data_shift_1['auto_engine'] = data_shift_1['auto_year']
data_shift_1['auto_year'] = data_shift_1['auto_owners']
data_shift_1['auto_owners'] = "Нет данных"
data_shift_1

,Type,auto_price,auto_year,auto_owners,auto_guarantee,auto_mileage,auto_engine,auto_transmission,auto_drive_unit,auto_wheel,auto_carcase,auto_color,link,guarantee


In [906]:
data = pd.concat([data, data_shift_1])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2159 entries, 0 to 2130
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Type               2159 non-null   object
 1   auto_price         2159 non-null   object
 2   auto_year          2159 non-null   object
 3   auto_owners        2159 non-null   object
 4   auto_guarantee     2159 non-null   object
 5   auto_mileage       2159 non-null   object
 6   auto_engine        2159 non-null   object
 7   auto_transmission  2159 non-null   object
 8   auto_drive_unit    2159 non-null   object
 9   auto_wheel         2159 non-null   object
 10  auto_carcase       2159 non-null   object
 11  auto_color         2159 non-null   object
 12  link               2159 non-null   object
 13  guarantee          2159 non-null   int64 
dtypes: int64(1), object(13)
memory usage: 253.0+ KB


Ok, missing data were handled

<a id='Removing_extra_spaces'></a>
### Removing extra spaces

In [907]:
data['auto_carcase'] = data['auto_carcase'].str.strip()
data['auto_color'] = data['auto_color'].str.strip()
data['auto_wheel'] = data['auto_wheel'].str.strip()
data['auto_drive_unit'] = data['auto_drive_unit'].str.strip()
data['auto_transmission'] = data['auto_transmission'].str.strip()
data['auto_price'] = data['auto_price'].str.strip()
data['auto_year'] = data['auto_year'].str.strip()
data['auto_owners'] = data['auto_owners'].str.strip()
data['auto_guarantee'] = data['auto_guarantee'].str.strip()
data['auto_mileage'] = data['auto_mileage'].str.strip()
data['auto_engine'] = data['auto_engine'].str.strip()

In [908]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2159 entries, 0 to 2130
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Type               2159 non-null   object
 1   auto_price         2159 non-null   object
 2   auto_year          2159 non-null   object
 3   auto_owners        2159 non-null   object
 4   auto_guarantee     2159 non-null   object
 5   auto_mileage       2159 non-null   object
 6   auto_engine        2159 non-null   object
 7   auto_transmission  2159 non-null   object
 8   auto_drive_unit    2159 non-null   object
 9   auto_wheel         2159 non-null   object
 10  auto_carcase       2159 non-null   object
 11  auto_color         2159 non-null   object
 12  link               2159 non-null   object
 13  guarantee          2159 non-null   int64 
dtypes: int64(1), object(13)
memory usage: 253.0+ KB


<a id='Shifting_data_on_right_columns'></a>
### Shifting data on right columns

In [909]:
tmp = ['до\n                  12.2023', 'до\n                  07.2024',
       'до\n                  11.2024', 'до\n                  01.2027',
       'до\n                  08.2025', 'до\n                  10.2024',
       'до\n                  11.2023']

In [910]:
owners = data.loc[data['auto_owners'].isin(tmp)]
data = data.drop(data[data['auto_owners'].isin(tmp)].index)
owners.shape, data.shape

((7, 14), (2152, 14))

In [911]:
change_mil_eng = data[data['auto_owners']=='Нет данных']
data = data.drop(data[data['auto_owners']=='Нет данных'].index)

In [912]:
owners.shape, change_mil_eng.shape, data.shape

((7, 14), (10, 14), (2142, 14))

In [913]:
owners['auto_guarantee'] = owners['auto_owners']

In [914]:
owners['auto_owners'] = "Нет данных"

In [915]:
change_mil_eng['auto_guarantee'] = change_mil_eng['auto_mileage'] 
change_mil_eng['auto_mileage'] = change_mil_eng['auto_engine']
change_mil_eng['auto_engine'] = change_mil_eng['auto_guarantee']
change_mil_eng['auto_guarantee'] = "Нет гарантии"

In [916]:
data = pd.concat([data, owners])
data = pd.concat([data, change_mil_eng])
data.shape

(2159, 14)

In [917]:
data.auto_owners.unique()

array(['3                владельца', '4                владельца',
       '1                владелец', '6                владельцев',
       '2                владельца', '9                владельцев',
       '5                владельцев', '7                владельцев',
       '8                владельцев', '16                владельцев',
       '11                владельцев', '10                владельцев',
       'Нет данных'], dtype=object)

In [918]:
data.auto_guarantee.unique()

array(['до\n                  02.2024', 'до\n                  01.2024',
       'до\n                  11.2023', 'до\n                  12.2023',
       'до\n                  10.2024', 'до\n                  03.2022',
       'до\n                  04.2023', 'до\n                  01.2023',
       'до\n                  06.2024', 'до\n                  11.2025',
       'до\n                  10.2023', 'до\n                  07.2022',
       'до\n                  05.2024', 'до\n                  05.2022',
       'до\n                  03.2026', 'до\n                  08.2023',
       'до\n                  08.2022', 'до\n                  10.2022',
       'до\n                  02.2023', 'до\n                  01.2026',
       'до\n                  06.2023', 'до\n                  11.2022',
       'до\n                  12.2026', 'до\n                  06.2022',
       'до\n                  03.2024', 'до\n                  08.2025',
       'до\n                  06.2026', 'до\n      

In [919]:
data.auto_mileage.unique()

array(['133 499 км', '148 558 км', '28 022 км', ..., '51 917 км', '40 км',
       '67 539 км'], dtype=object)

In [920]:
data.auto_engine.unique()

array(['2.0 л /\n              150 л.с. /\n              Бензин',
       '2.5 л /\n              208 л.с. /\n              Бензин',
       '5.6 л /\n              405 л.с. /\n              Бензин',
       '2.0 л /\n              144 л.с. /\n              Бензин',
       '2.0 л /\n              170 л.с. /\n              Бензин',
       '1.6 л /\n              122 л.с. /\n              Бензин',
       '3.0 л /\n              249 л.с. /\n              Дизель',
       '1.6 л /\n              115 л.с. /\n              Бензин',
       '2.4 л /\n              140 л.с. /\n              Бензин',
       '2.5 л /\n              169 л.с. /\n              Бензин',
       '1.6 л /\n              102 л.с. /\n              Бензин',
       '1.6 л /\n              114 л.с. /\n              Бензин',
       '2.2 л /\n              190 л.с. /\n              Дизель',
       '4.4 л /\n              333 л.с. /\n              Бензин',
       '1.6 л /\n              105 л.с. /\n              Бензин',
       '2.

In [921]:
data.head()

,Type,auto_price,auto_year,auto_owners,auto_guarantee,auto_mileage,auto_engine,auto_transmission,auto_drive_unit,auto_wheel,auto_carcase,auto_color,link,guarantee
0,\n Mazda CX-5 АКПП 2015 с пробегом 133 ...,1 499 000 ₽,2015,3 владельца,до\n 02.2024,133 499 км,2.0 л /\n 150 л.с. /\n ...,Автоматическая,Передний,Левый,Внедорожник 5 дв.,Красный,https://rolf-probeg.ru//spb/cars/mazda/cx-5/15...,1
1,\n Lexus IS АКПП 2008 с пробегом 148 55...,1 097 000 ₽,2008,4 владельца,до\n 01.2024,148 558 км,2.5 л /\n 208 л.с. /\n ...,Автоматическая,Задний,Левый,Седан,Светло-серо-голубой,https://rolf-probeg.ru//spb/cars/lexus/is/1608...,1
2,\n Infiniti QX80 АКПП 2021 с пробегом 2...,6 995 600 ₽,2021,1 владелец,до\n 02.2024,28 022 км,5.6 л /\n 405 л.с. /\n ...,Автоматическая,Полный,Левый,Внедорожник 5 дв.,Желтый,https://rolf-probeg.ru//spb/cars/infiniti/qx80...,1
3,\n Nissan Qashqai CVT 2016 с пробегом 1...,1 568 000 ₽,2016,1 владелец,до\n 11.2023,18 517 км,2.0 л /\n 144 л.с. /\n ...,Вариатор,Передний,Левый,Внедорожник 5 дв.,Белый,https://rolf-probeg.ru//spb/cars/nissan/qashqa...,1
4,\n Hyundai i40 АКПП 2015 с пробегом 174...,983 000 ₽,2015,3 владельца,до\n 12.2023,174 494 км,2.0 л /\n 150 л.с. /\n ...,Автоматическая,Передний,Левый,Седан,Коричневый,https://rolf-probeg.ru//spb/cars/hyundai/i40/1...,1


<a id="Split_auto_engine"></a>
### Split data in "auto_engine" into 3 columns

In [922]:
data[['Engine_displacement', 'Capacity', 'Fuel']] = data['auto_engine'].str.split(pat='/\n', expand=True)
data.head()

,Type,auto_price,auto_year,auto_owners,auto_guarantee,auto_mileage,auto_engine,auto_transmission,auto_drive_unit,auto_wheel,auto_carcase,auto_color,link,guarantee,Engine_displacement,Capacity,Fuel
0,\n Mazda CX-5 АКПП 2015 с пробегом 133 ...,1 499 000 ₽,2015,3 владельца,до\n 02.2024,133 499 км,2.0 л /\n 150 л.с. /\n ...,Автоматическая,Передний,Левый,Внедорожник 5 дв.,Красный,https://rolf-probeg.ru//spb/cars/mazda/cx-5/15...,1,2.0 л,150 л.с.,Бензин
1,\n Lexus IS АКПП 2008 с пробегом 148 55...,1 097 000 ₽,2008,4 владельца,до\n 01.2024,148 558 км,2.5 л /\n 208 л.с. /\n ...,Автоматическая,Задний,Левый,Седан,Светло-серо-голубой,https://rolf-probeg.ru//spb/cars/lexus/is/1608...,1,2.5 л,208 л.с.,Бензин
2,\n Infiniti QX80 АКПП 2021 с пробегом 2...,6 995 600 ₽,2021,1 владелец,до\n 02.2024,28 022 км,5.6 л /\n 405 л.с. /\n ...,Автоматическая,Полный,Левый,Внедорожник 5 дв.,Желтый,https://rolf-probeg.ru//spb/cars/infiniti/qx80...,1,5.6 л,405 л.с.,Бензин
3,\n Nissan Qashqai CVT 2016 с пробегом 1...,1 568 000 ₽,2016,1 владелец,до\n 11.2023,18 517 км,2.0 л /\n 144 л.с. /\n ...,Вариатор,Передний,Левый,Внедорожник 5 дв.,Белый,https://rolf-probeg.ru//spb/cars/nissan/qashqa...,1,2.0 л,144 л.с.,Бензин
4,\n Hyundai i40 АКПП 2015 с пробегом 174...,983 000 ₽,2015,3 владельца,до\n 12.2023,174 494 км,2.0 л /\n 150 л.с. /\n ...,Автоматическая,Передний,Левый,Седан,Коричневый,https://rolf-probeg.ru//spb/cars/hyundai/i40/1...,1,2.0 л,150 л.с.,Бензин


In [923]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2159 entries, 0 to 2130
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Type                 2159 non-null   object
 1   auto_price           2159 non-null   object
 2   auto_year            2159 non-null   object
 3   auto_owners          2159 non-null   object
 4   auto_guarantee       2159 non-null   object
 5   auto_mileage         2159 non-null   object
 6   auto_engine          2159 non-null   object
 7   auto_transmission    2159 non-null   object
 8   auto_drive_unit      2159 non-null   object
 9   auto_wheel           2159 non-null   object
 10  auto_carcase         2159 non-null   object
 11  auto_color           2159 non-null   object
 12  link                 2159 non-null   object
 13  guarantee            2159 non-null   int64 
 14  Engine_displacement  2159 non-null   object
 15  Capacity             2159 non-null   object
 16  Fuel  

In [924]:
#data.Fuel.unique()
data[data['Fuel'].isnull()]

,Type,auto_price,auto_year,auto_owners,auto_guarantee,auto_mileage,auto_engine,auto_transmission,auto_drive_unit,auto_wheel,auto_carcase,auto_color,link,guarantee,Engine_displacement,Capacity,Fuel
1144,\n Volkswagen Multivan 2006 с пробегом...,1 118 950 ₽,2006,4 владельца,до\n 01.2024,369 664 км,0.0 л /\n 0 л.с. /,,,Левый,Минивэн,Черный,https://rolf-probeg.ru//spb/cars/volkswagen/mu...,1,0.0 л,0 л.с. /,None
1986,\n Mercedes Maybach S-Класс 2016 с про...,7 650 000 ₽,2016,3 владельца,Нет гарантии,35 507 км,0.0 л /\n 0 л.с. /,,,Левый,Седан,Белый,https://rolf-probeg.ru//spb/cars/mercedes/mayb...,1,0.0 л,0 л.с. /,None


Ok, there are 2 eows with empty columns related to engine.<br>
Links are broken, so we can not verify info.<br>
Let's just remove these 2 items.

In [925]:
data = data.dropna(subset=['Fuel'])

In [926]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2157 entries, 0 to 2130
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Type                 2157 non-null   object
 1   auto_price           2157 non-null   object
 2   auto_year            2157 non-null   object
 3   auto_owners          2157 non-null   object
 4   auto_guarantee       2157 non-null   object
 5   auto_mileage         2157 non-null   object
 6   auto_engine          2157 non-null   object
 7   auto_transmission    2157 non-null   object
 8   auto_drive_unit      2157 non-null   object
 9   auto_wheel           2157 non-null   object
 10  auto_carcase         2157 non-null   object
 11  auto_color           2157 non-null   object
 12  link                 2157 non-null   object
 13  guarantee            2157 non-null   int64 
 14  Engine_displacement  2157 non-null   object
 15  Capacity             2157 non-null   object
 16  Fuel  

<a id="Formating_columns"></a>
### Formating columns (Dtype, creating new columns, etc.)

In [927]:
#Formating column with price
data[['Price', 'Currency']] = data.auto_price.str.split(pat='                ', expand=True)
data.Price = data.Price.str.replace(' ', '')
data = data.astype({'Price': 'int64'})

#Formating column Engine_displacement
data.Engine_displacement = data.Engine_displacement.str.replace(' л ', '')
data = data.astype({'Engine_displacement': 'float64'})

#Formating column Capacity
data.Capacity = data.Capacity.str.replace(' л.с.', '')
data = data.astype({'Engine_displacement': 'int64'})

#Formating column owners
data[['owners', '1']] = data['auto_owners'].str.split(pat="                ", expand=True)
data = data.drop(['1'], axis=1)
data['owners'] = [None if x == "Нет данных" else x for x in data['owners']]

#Formating column auto_mileage
data['auto_mileage'] = data['auto_mileage'].str.replace(' км', '')
data['auto_mileage'] = data['auto_mileage'].str.replace(' ', '')
data['auto_mileage'] = data['auto_mileage'].astype('int64')

#Creating columns with brand and model
data[['htpp', 'emp', 'web', 'emp1', 'city', 'type', 'brand', 'model', 'id', 'em2']] = data['link'].str.split(pat='/', expand=True)


In [928]:
data = data.drop(['htpp', 'emp', 'web', 'emp1', 'city', 'type', 'em2'], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2157 entries, 0 to 2130
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Type                 2157 non-null   object
 1   auto_price           2157 non-null   object
 2   auto_year            2157 non-null   object
 3   auto_owners          2157 non-null   object
 4   auto_guarantee       2157 non-null   object
 5   auto_mileage         2157 non-null   int64 
 6   auto_engine          2157 non-null   object
 7   auto_transmission    2157 non-null   object
 8   auto_drive_unit      2157 non-null   object
 9   auto_wheel           2157 non-null   object
 10  auto_carcase         2157 non-null   object
 11  auto_color           2157 non-null   object
 12  link                 2157 non-null   object
 13  guarantee            2157 non-null   int64 
 14  Engine_displacement  2157 non-null   int64 
 15  Capacity             2157 non-null   object
 16  Fuel  

In [929]:
data.head()

,Type,auto_price,auto_year,auto_owners,auto_guarantee,auto_mileage,auto_engine,auto_transmission,auto_drive_unit,auto_wheel,...,guarantee,Engine_displacement,Capacity,Fuel,Price,Currency,owners,brand,model,id
0,\n Mazda CX-5 АКПП 2015 с пробегом 133 ...,1 499 000 ₽,2015,3 владельца,до\n 02.2024,133499,2.0 л /\n 150 л.с. /\n ...,Автоматическая,Передний,Левый,...,1,2,150,Бензин,1499000,₽,3,mazda,cx-5,15905191
1,\n Lexus IS АКПП 2008 с пробегом 148 55...,1 097 000 ₽,2008,4 владельца,до\n 01.2024,148558,2.5 л /\n 208 л.с. /\n ...,Автоматическая,Задний,Левый,...,1,2,208,Бензин,1097000,₽,4,lexus,is,16085241
2,\n Infiniti QX80 АКПП 2021 с пробегом 2...,6 995 600 ₽,2021,1 владелец,до\n 02.2024,28022,5.6 л /\n 405 л.с. /\n ...,Автоматическая,Полный,Левый,...,1,5,405,Бензин,6995600,₽,1,infiniti,qx80,16072132
3,\n Nissan Qashqai CVT 2016 с пробегом 1...,1 568 000 ₽,2016,1 владелец,до\n 11.2023,18517,2.0 л /\n 144 л.с. /\n ...,Вариатор,Передний,Левый,...,1,2,144,Бензин,1568000,₽,1,nissan,qashqai,14549428
4,\n Hyundai i40 АКПП 2015 с пробегом 174...,983 000 ₽,2015,3 владельца,до\n 12.2023,174494,2.0 л /\n 150 л.с. /\n ...,Автоматическая,Передний,Левый,...,1,2,150,Бензин,983000,₽,3,hyundai,i40,14817855


In [930]:
data.brand.unique()

array(['mazda', 'lexus', 'infiniti', 'nissan', 'hyundai', 'audi', 'kia',
       'mercedes', 'ford', 'volvo', 'land-rover', 'bmw', 'skoda',
       'genesis', 'volkswagen', 'jaguar', 'renault', 'toyota', 'peugeot',
       'mitsubishi', 'chery', 'honda', 'cadillac', 'subaru', 'lada--vaz-',
       'opel', 'porsche', 'chevrolet', 'suzuki', 'citroen', 'mini',
       'jeep', 'lifan', 'ssangyong', 'haval', 'seat', 'geely', 'fiat',
       'dodge', 'uaz', 'chrysler', 'datsun', 'pontiac', 'dongfeng', 'izh',
       'bentley', 'saab', 'alfa-romeo', 'maserati'], dtype=object)

In [931]:
data['brand_model'] = data['brand'] + "_" + data['model']
#df.apply(lambda row: fuzz.ratio(row['name'], row['name_2']), axis = 1)

In [932]:
data.head()

,Type,auto_price,auto_year,auto_owners,auto_guarantee,auto_mileage,auto_engine,auto_transmission,auto_drive_unit,auto_wheel,...,Engine_displacement,Capacity,Fuel,Price,Currency,owners,brand,model,id,brand_model
0,\n Mazda CX-5 АКПП 2015 с пробегом 133 ...,1 499 000 ₽,2015,3 владельца,до\n 02.2024,133499,2.0 л /\n 150 л.с. /\n ...,Автоматическая,Передний,Левый,...,2,150,Бензин,1499000,₽,3,mazda,cx-5,15905191,mazda_cx-5
1,\n Lexus IS АКПП 2008 с пробегом 148 55...,1 097 000 ₽,2008,4 владельца,до\n 01.2024,148558,2.5 л /\n 208 л.с. /\n ...,Автоматическая,Задний,Левый,...,2,208,Бензин,1097000,₽,4,lexus,is,16085241,lexus_is
2,\n Infiniti QX80 АКПП 2021 с пробегом 2...,6 995 600 ₽,2021,1 владелец,до\n 02.2024,28022,5.6 л /\n 405 л.с. /\n ...,Автоматическая,Полный,Левый,...,5,405,Бензин,6995600,₽,1,infiniti,qx80,16072132,infiniti_qx80
3,\n Nissan Qashqai CVT 2016 с пробегом 1...,1 568 000 ₽,2016,1 владелец,до\n 11.2023,18517,2.0 л /\n 144 л.с. /\n ...,Вариатор,Передний,Левый,...,2,144,Бензин,1568000,₽,1,nissan,qashqai,14549428,nissan_qashqai
4,\n Hyundai i40 АКПП 2015 с пробегом 174...,983 000 ₽,2015,3 владельца,до\n 12.2023,174494,2.0 л /\n 150 л.с. /\n ...,Автоматическая,Передний,Левый,...,2,150,Бензин,983000,₽,3,hyundai,i40,14817855,hyundai_i40


In [933]:
data[['emp','guarantee_date']] = data['auto_guarantee'].str.split(pat='                  ', expand=True)

In [934]:
data = data.drop(['emp'], axis=1)

In [935]:
data.guarantee_date.unique()

array(['02.2024', '01.2024', '11.2023', '12.2023', '10.2024', '03.2022',
       '04.2023', '01.2023', '06.2024', '11.2025', '10.2023', '07.2022',
       '05.2024', '05.2022', '03.2026', '08.2023', '08.2022', '10.2022',
       '02.2023', '01.2026', '06.2023', '11.2022', '12.2026', '06.2022',
       '03.2024', '08.2025', '06.2026', '09.2023', '04.2022', '07.2023',
       '09.2022', '04.2026', '12.2022', '12.2025', '02.2022', '02.2025',
       '03.2025', '12.2024', '05.2023', '04.2024', '02.2026', '09.2025',
       '03.2023', '09.2024', '07.2024', '08.2026', '07.2026', '07.2025',
       '09.2026', '04.2025', '10.2025', '01.2027', '08.2024', '05.2026',
       '10.2026', '11.2024', '05.2025', '06.2025', '01.2025', '11.2026',
       None], dtype=object)

In [936]:
data[['guarantee_month', 'guarantee_year']] = data['guarantee_date'].str.split(pat='.', expand=True)

In [937]:
data['guarantee_year'] = [0 if x is None else x for x in data['guarantee_year']]
data['guarantee_month'] = [0 if x is None else x for x in data['guarantee_month']]

In [938]:
data.guarantee_year.unique()

array(['2024', '2023', '2022', '2025', '2026', '2027', 0], dtype=object)

In [939]:
data = data.astype({'guarantee_year': 'int64', 'guarantee_month':'int64', 'auto_year': 'int64'})

In [940]:
data = data.drop(['auto_price', 'auto_owners', 'auto_guarantee', 'auto_engine', 'Currency'], axis=1)
data.head()

,Type,auto_year,auto_mileage,auto_transmission,auto_drive_unit,auto_wheel,auto_carcase,auto_color,link,guarantee,...,Fuel,Price,owners,brand,model,id,brand_model,guarantee_date,guarantee_month,guarantee_year
0,\n Mazda CX-5 АКПП 2015 с пробегом 133 ...,2015,133499,Автоматическая,Передний,Левый,Внедорожник 5 дв.,Красный,https://rolf-probeg.ru//spb/cars/mazda/cx-5/15...,1,...,Бензин,1499000,3,mazda,cx-5,15905191,mazda_cx-5,02.2024,2,2024
1,\n Lexus IS АКПП 2008 с пробегом 148 55...,2008,148558,Автоматическая,Задний,Левый,Седан,Светло-серо-голубой,https://rolf-probeg.ru//spb/cars/lexus/is/1608...,1,...,Бензин,1097000,4,lexus,is,16085241,lexus_is,01.2024,1,2024
2,\n Infiniti QX80 АКПП 2021 с пробегом 2...,2021,28022,Автоматическая,Полный,Левый,Внедорожник 5 дв.,Желтый,https://rolf-probeg.ru//spb/cars/infiniti/qx80...,1,...,Бензин,6995600,1,infiniti,qx80,16072132,infiniti_qx80,02.2024,2,2024
3,\n Nissan Qashqai CVT 2016 с пробегом 1...,2016,18517,Вариатор,Передний,Левый,Внедорожник 5 дв.,Белый,https://rolf-probeg.ru//spb/cars/nissan/qashqa...,1,...,Бензин,1568000,1,nissan,qashqai,14549428,nissan_qashqai,11.2023,11,2023
4,\n Hyundai i40 АКПП 2015 с пробегом 174...,2015,174494,Автоматическая,Передний,Левый,Седан,Коричневый,https://rolf-probeg.ru//spb/cars/hyundai/i40/1...,1,...,Бензин,983000,3,hyundai,i40,14817855,hyundai_i40,12.2023,12,2023


In [941]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2157 entries, 0 to 2130
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Type                 2157 non-null   object
 1   auto_year            2157 non-null   int64 
 2   auto_mileage         2157 non-null   int64 
 3   auto_transmission    2157 non-null   object
 4   auto_drive_unit      2157 non-null   object
 5   auto_wheel           2157 non-null   object
 6   auto_carcase         2157 non-null   object
 7   auto_color           2157 non-null   object
 8   link                 2157 non-null   object
 9   guarantee            2157 non-null   int64 
 10  Engine_displacement  2157 non-null   int64 
 11  Capacity             2157 non-null   object
 12  Fuel                 2157 non-null   object
 13  Price                2157 non-null   int64 
 14  owners               2140 non-null   object
 15  brand                2157 non-null   object
 16  model 

In [946]:
data.owners = data.owners.fillna(0)
data.guarantee_date = data.guarantee_date.fillna(0)

In [947]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2157 entries, 0 to 2130
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Type                 2157 non-null   object
 1   auto_year            2157 non-null   int64 
 2   auto_mileage         2157 non-null   int64 
 3   auto_transmission    2157 non-null   object
 4   auto_drive_unit      2157 non-null   object
 5   auto_wheel           2157 non-null   object
 6   auto_carcase         2157 non-null   object
 7   auto_color           2157 non-null   object
 8   link                 2157 non-null   object
 9   guarantee            2157 non-null   int64 
 10  Engine_displacement  2157 non-null   int64 
 11  Capacity             2157 non-null   object
 12  Fuel                 2157 non-null   object
 13  Price                2157 non-null   int64 
 14  owners               2157 non-null   object
 15  brand                2157 non-null   object
 16  model 

In [948]:
data.head()

,Type,auto_year,auto_mileage,auto_transmission,auto_drive_unit,auto_wheel,auto_carcase,auto_color,link,guarantee,...,Fuel,Price,owners,brand,model,id,brand_model,guarantee_date,guarantee_month,guarantee_year
0,\n Mazda CX-5 АКПП 2015 с пробегом 133 ...,2015,133499,Автоматическая,Передний,Левый,Внедорожник 5 дв.,Красный,https://rolf-probeg.ru//spb/cars/mazda/cx-5/15...,1,...,Бензин,1499000,3,mazda,cx-5,15905191,mazda_cx-5,02.2024,2,2024
1,\n Lexus IS АКПП 2008 с пробегом 148 55...,2008,148558,Автоматическая,Задний,Левый,Седан,Светло-серо-голубой,https://rolf-probeg.ru//spb/cars/lexus/is/1608...,1,...,Бензин,1097000,4,lexus,is,16085241,lexus_is,01.2024,1,2024
2,\n Infiniti QX80 АКПП 2021 с пробегом 2...,2021,28022,Автоматическая,Полный,Левый,Внедорожник 5 дв.,Желтый,https://rolf-probeg.ru//spb/cars/infiniti/qx80...,1,...,Бензин,6995600,1,infiniti,qx80,16072132,infiniti_qx80,02.2024,2,2024
3,\n Nissan Qashqai CVT 2016 с пробегом 1...,2016,18517,Вариатор,Передний,Левый,Внедорожник 5 дв.,Белый,https://rolf-probeg.ru//spb/cars/nissan/qashqa...,1,...,Бензин,1568000,1,nissan,qashqai,14549428,nissan_qashqai,11.2023,11,2023
4,\n Hyundai i40 АКПП 2015 с пробегом 174...,2015,174494,Автоматическая,Передний,Левый,Седан,Коричневый,https://rolf-probeg.ru//spb/cars/hyundai/i40/1...,1,...,Бензин,983000,3,hyundai,i40,14817855,hyundai_i40,12.2023,12,2023
